# AI Engineering Drawing Inspector v2.0

**Single-File QC Pipeline**

Outputs:
1. `ResolvedPartIdentity.json`
2. `DrawingEvidence.json`
3. `DiffResult.json`
4. `QCReport.md`

In [ ]:
# Cell 1: Install Dependencies
!pip install -q pymupdf opencv-python-headless jsonschema pillow pytesseract
!pip install -q accelerate qwen-vl-utils bitsandbytes
!pip install -q git+https://github.com/huggingface/transformers
!apt-get install -y poppler-utils tesseract-ocr > /dev/null 2>&1
print("Dependencies installed!")

In [ ]:
# Cell 2: Imports and Configuration
import os, json, re, gc
import torch
import fitz
import numpy as np
from PIL import Image
from pathlib import Path
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime

# Configuration
DRAWING_PDF_PATH = ""
SOLIDWORKS_JSON_DIR = "sw_json_library"
OUTPUT_DIR = "qc_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# Cell 3: BOM-Robust JSON Loader
def load_json_robust(filepath) -> Tuple[Optional[Dict], Optional[str]]:
    """Load JSON with BOM handling. Tries: utf-8-sig, utf-8, latin-1"""
    filepath = Path(filepath)
    for enc in ['utf-8-sig', 'utf-8', 'latin-1']:
        try:
            with open(filepath, 'r', encoding=enc) as f:
                return json.load(f), None
        except UnicodeDecodeError:
            continue
        except json.JSONDecodeError as e:
            if 'BOM' in str(e) and enc == 'utf-8':
                continue
            return None, f"JSON error: {str(e)[:50]}"
        except Exception as e:
            return None, f"Error: {str(e)[:50]}"
    return None, "Failed all encodings"

# Test
print("load_json_robust defined")

In [ ]:
# Cell 4: PDF Rendering
@dataclass
class PageArtifact:
    pageIndex0: int
    page: int
    image: Image.Image
    width: int
    height: int
    dpi: int
    direct_text: Optional[str] = None

def render_pdf(pdf_path: str, dpi: int = 300) -> List[PageArtifact]:
    """Render first page of PDF to image."""
    artifacts = []
    doc = fitz.open(pdf_path)
    page = doc.load_page(0)
    zoom = dpi / 72.0
    pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom), alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    direct_text = page.get_text("text")
    
    artifacts.append(PageArtifact(
        pageIndex0=0, page=1, image=img,
        width=pix.width, height=pix.height, dpi=dpi,
        direct_text=direct_text if len(direct_text.strip()) > 10 else None
    ))
    doc.close()
    print(f"Rendered: {pix.width}x{pix.height}px")
    return artifacts

print("render_pdf defined")

In [ ]:
# Cell 5: SolidWorks JSON Library
@dataclass
class SwPartEntry:
    json_path: str
    part_number: str
    filename_stem: str = ""
    data: Dict[str, Any] = field(default_factory=dict)

class SwJsonLibrary:
    def __init__(self):
        self.by_part_number: Dict[str, SwPartEntry] = {}
        self.by_filename: Dict[str, SwPartEntry] = {}
        self.all_entries: List[SwPartEntry] = []
    
    def _normalize(self, s: str) -> str:
        return re.sub(r'[-\s_]', '', str(s or '')).lower()
    
    def load_from_directory(self, directory: str):
        json_files = list(Path(directory).glob("**/*.json"))
        print(f"Found {len(json_files)} JSON files")
        
        for jp in json_files:
            data, err = load_json_robust(jp)
            if data is None:
                continue
            pn = data.get('identity', {}).get('partNumber', '')
            entry = SwPartEntry(str(jp), pn, jp.stem, data)
            self.all_entries.append(entry)
            if pn:
                self.by_part_number[pn] = entry
                self.by_part_number[self._normalize(pn)] = entry
            self.by_filename[jp.stem] = entry
            self.by_filename[self._normalize(jp.stem)] = entry
        print(f"Loaded {len(self.all_entries)} files")
    
    def lookup(self, candidate: str) -> Optional[SwPartEntry]:
        if not candidate:
            return None
        norm = self._normalize(candidate)
        return self.by_part_number.get(candidate) or self.by_part_number.get(norm) or \
               self.by_filename.get(candidate) or self.by_filename.get(norm)

sw_library = SwJsonLibrary()
print("SwJsonLibrary defined")

In [ ]:
# Cell 6: Part Identity Resolution
PART_PATTERNS = [
    r'\b(\d{3}-\d{3}(?:-\d+)?)\b',
    r'\bP/?N[:\s]*([A-Z0-9-]+)\b',
]

@dataclass
class ResolvedPartIdentity:
    partNumber: str
    confidence: float
    source: str
    swJsonPath: Optional[str] = None

def resolve_part_identity(pdf_path: str, artifacts: List[PageArtifact], sw_lib: SwJsonLibrary) -> ResolvedPartIdentity:
    # Try filename
    stem = Path(pdf_path).stem.upper()
    for pat in PART_PATTERNS:
        m = re.search(pat, stem)
        if m:
            pn = m.group(1)
            entry = sw_lib.lookup(pn)
            if entry:
                return ResolvedPartIdentity(pn, 1.0, "filename+sw", entry.json_path)
            return ResolvedPartIdentity(pn, 0.9, "filename", None)
    
    # Try PDF text
    for art in artifacts:
        if art.direct_text:
            for pat in PART_PATTERNS:
                m = re.search(pat, art.direct_text.upper())
                if m:
                    pn = m.group(1)
                    entry = sw_lib.lookup(pn)
                    if entry:
                        return ResolvedPartIdentity(pn, 0.8, "pdf+sw", entry.json_path)
    
    return ResolvedPartIdentity(stem, 0.3, "fallback", None)

print("resolve_part_identity defined")